In [1]:
import os
import openai
import dotenv
import json
import fitz

In [2]:
def extract_information(text, functions):
    #call the LLM model with functions and the user's input
    messages=[{"role": "user", "content": text}]
    
    completion = openai.chat.completions.create(
    model="gpt-4", 
    messages=messages,
    functions =functions,
    function_call={"name":"extract_attributes"}
    )
    choice = completion.choices[0]
    extracted_data = choice.message.function_call.arguments
    
    print(json.dumps(json.loads(extracted_data), indent=4))
    return extracted_data
    

In [3]:
if __name__=='__main__':
    try:
        doc = fitz.open('./data/nw1.pdf')
    except IOError:
        print ("Could not read file:")

    text_to_analyze =""
    for page in doc:
        text_to_analyze += page.get_text()

    #define a API calling function: Property
    functions1=[
    {
            "name": "extract_attributes",
            "description": "Extract property, Lease details, Demographics mentioned in the text",
            "parameters": {
                "type": "object",
                "properties": {
                    "Property": {
                        "type": "array",
                        "items": {
                            "type": "object",
                            "properties": {
                                "Address": {
                                    "type": "string",
                                    "description": "Address of the property"
                                },
                                "RentableArea": {
                                    "type": "number",
                                    "description": "Rentable area of the property"
                                },
                                "NetOperatingIncome": {
                                    "type": "number",
                                    "description": "Net operating income of the property"
                                },
                                "LandArea": {
                                    "type": "number",
                                    "description": "Land area of the property"
                                },
                                "CapRate": {
                                    "type": "number",
                                    "description": "Cap rate of the property"
                                },
                                "YearBuilt": {
                                    "type": "string",
                                    "description": "Year built of the property"
                                },
                                "SalePrice": {
                                    "type": "number",
                                    "description": "Sale price of the property"
                                },
                                "Tenancy": {
                                    "type": "string",
                                    "description": "Tenancy of the property"
                                },
                                "ExpirationDate": {
                                    "type": "string",
                                    "description": "Expiration date of the property"
                                },
                                "TaxKey": {
                                    "type": "string",
                                    "description": "Tax key of the property"
                                },
                                "Taxes2021": {
                                    "type": "number",
                                    "description": "Taxes 2021 of the property"
                                },
                                "Municipality": {
                                    "type": "string",
                                    "description": "Municipality of the property"
                                }                                
                            }, 
                            "required": ["Address", "RentableArea","NetOperatingIncome","LandArea","CapRate","YearBuilt","SalePrice","Tenancy","ExpirationDate","TaxKey","Taxes2021","Municipality"]
                        }
                    }
                },
                "required": ["Property"],
            },
        },
    ]

    #define a API calling function: LeaseDetails
    functions2=[
    {
            "name": "extract_attributes",
            "description": "Extract property and Lease details mentioned in the text",
            "parameters": {
                "type": "object",
                "properties": {
                    "LeaseDetails": {
                        "type": "array",
                        "items": {
                            "type": "object",
                            "properties": {
                                "AnnualBaseRent": {
                                    "type": "number",
                                    "description": "Annual Base Rent of the property"
                                },
                                "CommonAreaMaintenance": {
                                    "type": "number",
                                    "description": "Common Area Maintenance of the property"
                                },
                                "Insurance": {
                                    "type": "string",
                                    "description": "Insurance of the property"
                                },
                                "RealEstateTaxes": {
                                    "type": "number",
                                    "description": "Real Estate Tax of the property"
                                },
                                "ExpirationDate": {
                                    "type": "string",
                                    "description": "Expiration Date of the property"
                                },
                                "BaseRentIncrease": {
                                    "type": "number",
                                    "description": "Base Rent Increase of the property"
                                },
                                "Options": {
                                    "type": "string",
                                    "description": "Options of the property"
                                },
                                "Grantor": {
                                    "type": "string",
                                    "description": "Grantor of the property"
                                }
                            },

                            "required": ["AnnualBaseRent", "CommonAreaMaintenance","Insurance","RealEstateTaxes","ExpirationDate","BaseRentIncrease","Options","Grantor"]
                        }
                    }
                },
                "required": ["LeaseDetails"],
            },
        },
    ]

    #extract Property attributes from text
    extracted_data1 =extract_information(text_to_analyze, functions1)
    try: 
        json_object1 = json.dumps(extracted_data1, indent=4)
        with open("Property.json", "w") as file:
            file.write(json_object1)
    except (json.JSONDecodeError, IndexError):
        pass

    #extract leaseDetails from text
    extracted_data2 =extract_information(text_to_analyze, functions2)
    try: 
        json_object2 = json.dumps(extracted_data2, indent=4)
        with open("LeaseDetails.json", "w") as file:
            file.write(json_object2)
    except (json.JSONDecodeError, IndexError):
        pass
    

{
    "Property": [
        {
            "Address": "1401 N Casaloma Dr, Appleton, WI",
            "RentableArea": 7609,
            "NetOperatingIncome": 102000,
            "LandArea": 1.32,
            "CapRate": 6.75,
            "YearBuilt": "1975",
            "SalePrice": 1511111,
            "Tenancy": "Single",
            "ExpirationDate": "12-31-2033",
            "TaxKey": "101086805",
            "Taxes2021": 12099.79,
            "Municipality": "Town of Grand Chute"
        },
        {
            "Address": "W3171 Springfield Dr, Appleton, WI",
            "RentableArea": 6528,
            "NetOperatingIncome": 102000,
            "LandArea": 0.72,
            "CapRate": 6.75,
            "YearBuilt": "2017",
            "SalePrice": 1511111,
            "Tenancy": "Single",
            "ExpirationDate": "12-31-2033",
            "TaxKey": "030264601",
            "Taxes2021": 13444.08,
            "Municipality": "Town of Buchanan"
        }
    ]
}
{
    "LeaseDeta